## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
Sunny, Rainy, Sunny: 0.4*(0.6)*0.4*(0.1)*0.3*(0.6) = 0.001728
Sunny, Rainy, Rainy: 0.4*(0.6)*0.4*(0.1)*0.7*(0.1) = 0.000672
Sunny, Sunny, Rainy: 0.4*(0.6)*0.6*(0.6)*0.4*(0.1) = 0.003456
Rainy, Rainy, Sunny: 0.6*(0.1)*0.7*(0.1)*0.3*(0.6) = 0.000756
Rainy, Sunny, Rainy: 0.6*(0.1)*0.3*(0.6)*0.4*(0.1) = 0.000432
Rainy, Rainy, Rainy: 0.6*(0.1)*0.7*(0.1)*0.7*(0.1) = 0.000294
###<your answers>###

最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [153]:
import pandas as pd

def vit(n,init_p,trans_p,emission_p,obs,trans = False):

    def transfer(item):  
        lst = []
        for i in range(len(item)):
            lst.append(item[i:i+2] if i+2 <=len(item)  else '')
        return pd.Series(lst[:-1])
 
    init_denote = list(init_p.keys())
    if n == 0:
        return pd.Series(init_denote)
    else:
        x = vit(n-1,init_p,trans_p,emission_p,obs,trans = False).map(lambda x:x+init_denote[0])
        for i in range(len(init_denote)):
            try:
                x = x.append(vit(n-1,init_p,trans_p,emission_p,obs,trans = False).map(lambda x:x+init_denote[i+1]))
            except:
                continue
                
        if not trans:
            return x
        else:
            final_p =  x.map(transfer).map(lambda x:x.map(lambda y:trans_p[y]).product()*init_p[x[0][0]])
            emission = x.map(lambda x:[i for i in x]).map(lambda x:pd.Series([emission_p[x[i]][obs[i]] for i in range(len(x))]).product())
            dict_ = dict(zip(x,round(final_p*emission,6)))
            return dict_, (max(dict_,key=dict_.get), dict_[max(dict_,key=dict_.get)]) 

transition_prob = {'SS':0.6,'SR':0.4,'RS':0.3,'RR':0.7}
init_prob = {'S':0.4,'R':0.6}
emission_prob = {'S': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
              'R': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
observations = ['dry','dry','dry']

path_prob_dict,most_likely_path = vit(2,init_prob,transition_prob,emission_prob,observations,trans=True)

print(path_prob_dict)
print(most_likely_path)


{'SSS': 0.031104, 'RSS': 0.003888, 'SRS': 0.001728, 'RRS': 0.000756, 'SSR': 0.003456, 'RSR': 0.000432, 'SRR': 0.000672, 'RRR': 0.000294}
('SSS', 0.031104)
